### Extra. Berekenen van de cosinusgelijkenis tussen documenten (document vectors)

Het doel van dit script is om te berekenen in hoeverre documenten binnen een corpus met elkaar overeenkomen. Dit wordt gedaan aan de hand van de zogeheten cosinusgelijkenis. Twee identieke documenten krijgen een waarde van 1 toegewezenn. Twee documenten die veel samenhang vertonen krijgen een waarde die dichtbij de 1 ligt, terwijl twee documenten met minder samenhang een waarde krijgen die dichterbij de 0 ligt. 

Op basis van de uitkomst van de cosinusgelijkenis kunnen identieke documenten uit een corpus worden gehaald, ook al hebben ze afwijkende bestandsnamen. Ook kunnen nagenoeg identieke documenten worden gevonden, hierbij blijft de vraag wanneer een nagenoeg identiek document verwijderd kan worden uit een corpus en wanneer niet. Hiervoor bestaan geen duidelijke richtlijnen.

Om het script te laten werken, is het noodzakelijk om de Natural Language Toolkit (NLTK), Numpy, Pandas, Scipy, en Scikit-Learn te installeren. Deze kunnen via de prompt geïnstalleerd worden met de onderstaande commandos:

- `pip install nltk`
- `pip install numpy`
- `pip install pandas`
- `pip install scipy`
- `pip install -U scikit-learn`

Documentatie:
- NLTK: https://www.nltk.org/index.html
- Numpy: https://numpy.org/
- Pandas: https://pandas.pydata.org/
- Scipy: https://scipy.org/ 
- Scikit-learn: https://scikit-learn.org/stable/install.html

Voor meer informatie over de cosinusgelijkenis, zie: https://www.youtube.com/watch?v=XartD5Z4XZM

Dit script komt uit: Aman Kedia & Mayank Rasu (2020) *Hands-On Python Natural Language Processing*, p. 92-95 

#### Stap 1: Importeren van de benodigde python biblioteken 

In [ ]:
import nltk
import os
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#### Stap 2: Laden van het corpus

Om dit script te laten draaien, zijn twee onderdelen nodig. Deze twee onderdelen worden gecreëerd in de onderstaande celblokjes. Ten eerste moeten we een corpus definiëren met de teksten uit de json. Ten tweede moeten we een index samenstellen met de bestandsnamen, zodat deze later uit het corpus kunnen worden gehaald als de identieke of nagenoeg identieke documenten zijn geïdentificeerd. Het is dus belangrijk om te checken of de documentnamen corresponderen met de tekst. 

In [ ]:
#teksten uit de json laden
filepath = '' #VUL IN: plaats tussen de aanhalingstekens het pad naar het json-bestand dat je in Hst 3 gemaakt hebt

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)
                                                                 
texts = load_data(filepath)["texts"] #lijst met alle teksten uit de json 
doc_id = load_data(filepath)["doc_id"] #lijst met alle bestandsnamen uit de json 

In [ ]:
#corpus samenstellen
corpus = pd.Series(texts, index=[doc_id]) #samenvoegen van lijst van teksten en index
corpus

#### Stap 3: preprocessing van de data

In de onderstaande celblokjes wordt de benodige preprocessing oftewel corpusopschoning uitgevoerd. De corpusopschoning zorgt ervoor dat de teksten kleiner worden en de berekening van de cosinusgelijkenissen minder vraagt van de computer. In de blokjes hoeft niets te worden aangepast of ingevuld, zolang er met behulp van de bovenstaande celblokjes eerst een corpus is samengesteld.

In [ ]:
#In de onderstaande celblokjes worden verschillende pre-processing taken gedefinieerd, die later tegelijkertijd toegepast kunnen worden

def text_clean(corpus, keep_list):
   
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            if word not in keep_list:
                p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
                p1 = p1.lower()
                qs.append(p1)
            else : qs.append(word)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

In [ ]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [ ]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [ ]:
def stopwords_removal(corpus):
    stop = set(stopwords.words('english'))
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [ ]:
#functie waarmee de bovenstaande preprocessing taken tegelijk kunnen worden uitgevoerd (opschoning, stemming of lemmatization, verwijderen van stopwoorden) 

def preprocess(corpus, keep_list, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    
    #middels de Boolean variabelen kan een keuze worden gemaakt om een bepaalde taak wel of niet uit te voeren. Er moet een keuze worden gemaakt 
    #tussen stemming en lemmatization. Allebei toepassen is nutteloos. Lemmatization heeft de voorkeur omdat hierbij ook rekening wordt gehouden 
    #met de functie/plaats van een woord in de zin.
    
    if cleaning == True:
        corpus = text_clean(corpus, keep_list)
    
    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else:
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        corpus = lemmatize(corpus)   
        
    if stemming == True:
        corpus = stem(corpus, stem_type)
    
    corpus = [' '.join(x) for x in corpus]        

    return corpus

In [ ]:
# Preprocessing van het corpus met gebruik van lemmatisatie en het verwijderen van stopwoorden (lemmatization = True, remove_stopwords = True)
preprocessed_corpus = preprocess(corpus, keep_list = [], stemming = False, stem_type = None,
                                lemmatization = True, remove_stopwords = True)
preprocessed_corpus[0] #eerste tekst uit tekst wordt geprint ter controle

#### Stap 4: Berekenen van de cosinusgelijkenis met TFIDF document vectors 

Nu het corpus is opgeschoond, kan de cosinusgelijkenis berekend worden. Dit wordt in de onderstaande celblokjes gedaan door eerst de documenten als vector te representeren en vervolgens de cosinusgelijkenis tussen elk document te berekenen. 

In [ ]:
def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    return np.dot(vector1, vector2) / (np.sqrt(np.sum(vector1**2)) * np.sqrt(np.sum(vector2**2)))

In [ ]:
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocessed_corpus)

In [ ]:
#hier printen we alleen de cosinusgelijkenis voor documenten die identiek zijn en documenten die voor tussen 95% en 99% procent overeenkomen, 
#zie het volgende celblokje om alle cosinusgelijkeniswaardes tussen alle documenten te printen

j_list = [] 

for i in range(tf_idf_matrix.shape[0]):
    for j in range(i + 1, tf_idf_matrix.shape[0]):
        if cosine_similarity(tf_idf_matrix.toarray()[i], tf_idf_matrix.toarray()[j]) >= 1:
            j_list.append(j)
            print("De cosinusgelijkenis tussen documenten ", i, "en", j, "is 1")
        if 0.95 < cosine_similarity(tf_idf_matrix.toarray()[i], tf_idf_matrix.toarray()[j]) < 1: #de waarde 0.95 kan eventueel worden aangepast 
            j_list.append(j)
            print("De cosinusgelijkenis tussen documenten ", i, "en", j, "is tussen 0.95 en 0.99")

print(j_list)

In [ ]:
#hier bereken we de cosinusgelijkenis voor alle documenten. Let op: een lange lijst met waardes wordt geprint.
for i in range(tf_idf_matrix.shape[0]):
    for j in range(i + 1, tf_idf_matrix.shape[0]):
        print("De cosinusgelijkenis tussen documenten ", i, "en", j, "is: ",
              cosine_similarity(tf_idf_matrix.toarray()[i], tf_idf_matrix.toarray()[j]))

In [ ]:
#printen van alleen de identieke documenten

identical_doc = []

for i in range(tf_idf_matrix.shape[0]):
    for j in range(i + 1, tf_idf_matrix.shape[0]):
        if cosine_similarity(tf_idf_matrix.toarray()[i], tf_idf_matrix.toarray()[j]) >= 1:
            identical_doc.append(j)
            print("De cosinusgelijkenis tussen documenten ", i, "en", j, "is 1")

#### Stap 5: Identieke & nagenoeg identieke documenten verwijderen uit het corpus

De uitkomst van het vorige celblokje kan nu gebruikt worden om een nieuw corpus te creeëren zonder de identieke en nagenoeg identieke documenten. Hiervoor moet de index en de lijst met originele tekst-bestanden 'getrimd' worden. Als alleen identieke documenten verwijderd moeten worden, vervang dan in de onderstaande celblokjes de lijst j_list met de lijst identical_doc en de lijst j_list1 met de lijst identical_doc1.

In [ ]:
print(j_list)

In [ ]:
#herhalingen uit lijst halen, we willen elk indexnummer slechts een keer in de lijst hebben
j_list1 = []

for i in j_list:
    if i not in j_list1:
        j_list1.append(i)

In [ ]:
print(j_list1)

In [ ]:
def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)

#waardes van j_list1 uit index verwijderen
delete_multiple_element(doc_id, j_list1)
#waardes van j_list1 uit lijst met documenten verwijderen
delete_multiple_element(texts, j_list1)


In [ ]:
#controleer of de lengte van documents en index overeenkomen
print(len(doc_id))
print(len(texts))

#### Stap 6: Nieuwe json samenstellen zonder identieke en nagenoeg identieke bestanden

In [ ]:
import json

path = '' + '/' #VUL IN: plaats tussen de eerste aanhalingstekens het pad naar de map waar je het json-bestand wilt bewaren
filename = '' + '.json' #VUL IN: plaats tussen de eerste aanhalingstekens de bestandsnaam die je aan het json-bestand wilt geven

lists = ['doc_id', 'texts']

data = {listname: globals()[listname] for listname in lists}
with open(path + filename, 'w') as outfile:  
    json.dump(data, outfile, indent=4)

#### Bonus: Kopiëren, verplaatsen of verwijderen van bepaalde bestanden met glob en shutil uit archief op basis van cosinusgelijkenis

Met de onderstaande cel blokjes kunnen de identieke en nagenoeg identieke bestanden uit het corpus worden gekopieerd (shutil.copy), verplaatst (shutil.move) of verwijderd (os.remove) worden. Hiervoor moeten we alle bestandsnamen uit de index verwijderen die we willen behouden.

Bronvermelding: https://thispointer.com/python-how-to-remove-files-by-matching-pattern-wildcards-certain-extensions-only/ 

In [ ]:
# Nieuwe index samenstellen met alle bestandsnamen
index2 = load_data(filepath)["doc_id"] 

In [ ]:
files_delete = sorted(j_list1) #vervang j_list1 met identical_doc1 als alleen identieke documenten verplaats, gekopieerd of verwijderd moeten worden. 

In [ ]:
files_all = list(range(X)) #VUL IN: vervang de X voor het totaal aantal bestanden in het corpus
files_keep = []

for i in files_all:
    if not i in files_delete:
        files_keep.append(i)
        
print(len(files_keep))

In [ ]:
#verwijderen van files_keep uit de index, deze willen we namelijk behouden in het archief 
def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)
            
delete_multiple_element(index2, files_keep)
print(len(index2))

Met het onderstaande script kun je de uit het archief te verwijderen bestanden kopiëren

In [ ]:
#shutil.copy = kopiëren van bestanden
import glob
import os
import shutil

path_copy = '' #VUL IN: plaats tussen de aanhalingstekens het pad naar de hoofdmap van het archief

dst_folder_copy = '' + '/' #VUL IN: plaats tussen de eerste aanhalingstekens het pad van de map waarnaar de bestanden gekopieerd moeten worden

for item in index2:
    files = glob.glob(path_copy + f"/**/{item.title()}", recursive = True)
    
    for file in files:
        file_name_copy = os.path.basename(file)
        shutil.copy(file, dst_folder_copy + file_name_copy) 
        print('Gekopieerd naar:', file)

Met het onderstaande script kun je de uit het archief te verwijderen bestanden verplaatsen

In [ ]:
#shutil.move = verplaatsen van bestanden
import glob
import os
import shutil

path_move = '' #VUL IN: plaats tussen de aanhalingstekens het pad naar de hoofdmap van het archief

dst_folder_move = '' + '/' #VUL IN: plaats tussen de eerste aanhalingstekens het pad naar de map waarnaar je de bestanden wilt verplaatsen

for item in index2:
    files = glob.glob(path_move + f"/**/{item.title()}", recursive = True)
    
    for file in files:
        file_name_move = os.path.basename(file)
        shutil.move(file, dst_folder_move + file_name_move) 
        print('Verplaatst naar:', file)

Met het onderstaande script kun je de uit het archief te verwijderen bestanden verwijderen

In [ ]:
#os.remove = let op: met os.remove worden bestanden verwijderd 
import glob
import os
import shutil

path_delete = '' #VUL IN: plaats tussen de aanhalingstekens het pad naar de hoofdmap van het archief

for item in index2:
    files = glob.glob(path_delete + f"/**/{item.title()}", recursive = True)
    
    for file in files:
        file_name = os.path.basename(file)
        os.remove(file)
        print('Verwijderd:', file)